In [4]:
import tifffile
import blosc
import numpy as np

blosc.set_nthreads(4)

royer1 = "/scratch/steinbac/sqeasy_data/xwing/droso-royer-may31-000005.tif"
xwingr = tifffile.imread(royer1)

haase1 = "/scratch/steinbac/sqeasy_data/xwing/000195-rhaase-20171129-fused-512x1024x92.tif"
xwingh = tifffile.imread(haase1)


In [5]:
print(xwingr.shape,xwingh.shape)

(108, 1352, 532) (92, 1024, 512)


# study of block size dependence in blosc

- varying the block size from 0.5,1,2,4,8 frames at a time to see change in compression ratio

In [7]:
import math
import time

smallr = xwingr[:,:200,:]
smallh = xwingh[:,:200,:]

sizes = {"smallr" : len(smallr.tobytes()), "smallh" : len(smallh.tobytes())}

bitshuffled_nbytes = {}

def scan_blocksize(data,_shuffle = blosc.BITSHUFFLE):
    value = []
    for f in [0.1,0.25,0.5,1,2,4,8]:
        blosc.set_blocksize( math.floor(f*data.shape[-1]*data.shape[-2]*2) )
        start = time.process_time()
        compressed = blosc.compress(data.tobytes(),shuffle=_shuffle,cname="lz4")
        end = time.process_time()
        
        value.append((f,len(compressed),end-start))
    return value
    
bitshuffled_nbytes["smallr"] = scan_blocksize(smallr)
bitshuffled_nbytes["smallh"] = scan_blocksize(smallh)

In [10]:
def summary(cdic,nbytes):
    print("%8s %10s %s"%("factor","outbytes","compression ratio"))
    for (k,v) in cdic.items():
        print(k)
        for item in v:
            ratio = nbytes[k]/item[1]
            bw = nbytes[k]/(1024*1024.*item[-1])
            print("%8f %10i"%(item[0],item[1]),ratio,bw," MB/s")
            
summary(bitshuffled_nbytes,sizes)

  factor   outbytes compression ratio
smallr
0.100000    4518718 5.086044316109127 217.62448883716283  MB/s
0.250000    4340482 5.294895820325945 207.31147723858675  MB/s
0.500000    4546011 5.0555091045754175 215.89189817968787  MB/s
1.000000    3481865 6.600600540227723 108.00340629113894  MB/s
2.000000    3473309 6.6168601756998875 107.62294009078424  MB/s
4.000000    3460662 6.641041511710765 105.80248687325266  MB/s
8.000000    3447474 6.66644621540293 105.2349121384035  MB/s
smallh
0.100000    4405217 4.27711052599679 106.75347732204749  MB/s
0.250000    4341645 4.339737587941897 109.76641261457607  MB/s
0.500000    4319646 4.361838909947713 111.08994533166046  MB/s
1.000000    4316498 4.365019976842339 109.56294465635213  MB/s
2.000000    4314466 4.367075786435679 106.94527350487064  MB/s
4.000000    4293839 4.388054605680371 103.1637813466253  MB/s
8.000000    4278461 4.403826516123438 103.99872048945329  MB/s


In [11]:
noshuffle_nbytes = {}
noshuffle_nbytes["smallr"] = scan_blocksize(smallr,blosc.NOSHUFFLE)
noshuffle_nbytes["smallh"] = scan_blocksize(smallh,blosc.NOSHUFFLE)
summary(noshuffle_nbytes,sizes)

  factor   outbytes compression ratio
smallr
0.100000    4518718 5.086044316109127 181.4593144185487  MB/s
0.250000    4340482 5.294895820325945 220.19997367950285  MB/s
0.500000    4546011 5.0555091045754175 219.53795573654244  MB/s
1.000000    4513353 5.092090071394814 214.02248249937907  MB/s
2.000000    4507745 5.098425044007591 215.6106555949238  MB/s
4.000000    4505606 5.100845480053072 218.0719983267917  MB/s
8.000000    4504560 5.102029942991102 222.86803208090257  MB/s
smallh
0.100000   11382028 1.6553816244345911 156.855552453919  MB/s
0.250000   11372836 1.656719572848848 163.621441898938  MB/s
0.500000    8339611 2.25929003163337 174.6027789814738  MB/s
1.000000    8326104 2.262955158859414 162.14362174208307  MB/s
2.000000    8323912 2.2635510803093544 179.8561871178973  MB/s
4.000000    8322422 2.26395633386531 180.1625399740249  MB/s
8.000000    8322001 2.264070864687471 179.26468094558135  MB/s
